In [15]:
import pandas as pd
import s3fs
import boto3
from io import StringIO # python3; python2: BytesIO 
from boto3.s3.transfer import TransferConfig
import torch
from transformers import *
import numpy as np
import re
from langdetect import detect
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer

In [132]:
pd.set_option('display.max_colwidth', -1)

In [23]:
import gensim
from gensim import models

In [36]:
import lda

In [6]:
val_input = pd.read_csv('s3://recsys-challenge-2020/val_input.csv')

In [7]:
languages = list(val_input.name.unique())

In [9]:
val_input.head()

,tweet_text,name
0,sm 사옥빌딩 앞에 있는 전광판차 사진들입니다 전광판차는 사옥 앞에서 오늘 아침 9...,ko
1,방송 막하지 말라던 최민호는 본인이 막히기 시작하는데 t co wqz58zzz5l,ko
2,rt lovablebh _ 0506 백현이 오늘 심각하게 unk 던지고 가만히 서서...,ko
3,sm 사옥빌딩 앞에 있는 전광판차 사진들입니다 전광판차는 사옥 앞에서 오늘 아침 9...,ko
4,sm 사옥빌딩 앞에 있는 전광판차 사진들입니다 전광판차는 사옥 앞에서 오늘 아침 9...,ko


In [10]:
val_input.sort_values(by='name', inplace=True)

In [ ]:
for language in languages:
    val_input.loc[val_input.name == language]

In [12]:
en_val_input = val_input.loc[val_input.name == 'en']

In [13]:
en_val_input.head()

,tweet_text,name
4955884,rt girlsreallyrule the world s biggest politic...,en
4955905,this is true the us unk interactions with fore...,en
4955906,it s a bit breezy that eccles cake and custard...,en
4955907,yo bizzel1 they got me too we all gonna take a...,en
4955908,rt uznmaki hello t co 0i2gh3tefh,en


In [18]:
fr_val_input = val_input.loc[val_input.name == 'fr']

In [133]:
fr_val_input.head()

,tweet_text,name
10507192,rt _ momo58 les fléchettes aux states c unk est un autre délire t co pwtg8pezun,fr
10507191,rt paulineolg moi tu me dis ça j unk te traite de mytho en te regardant droit dans les yeux faut que j unk adoucisse mon coeur un peu,fr
10507190,rt _ momo58 les fléchettes aux states c unk est un autre délire t co pwtg8pezun,fr
10507189,rt biboo _ r6 retrouvez moi dimanche 2 février à 19h30 dans l émission radio esix présenté par le magnifique r3siak unk je compte sur vous unk,fr
10507188,rt nayonek _ pour le dire plus poliment que certains tu as percé grâce à ta victimisation sur les réseaux et sur tes vidéos le fait que unk,fr


In [126]:
univectorizer = CountVectorizer(analyzer = "word", min_df = 0.001, ngram_range = (1,1), strip_accents= None)

In [127]:
unicorpus = univectorizer.fit_transform(fr_val_input["tweet_text"])

In [128]:
unigrams = univectorizer.get_feature_names()

In [129]:
unigrams

['00',
 '000',
 '01',
 '02',
 '10',
 '100',
 '1070',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '17h',
 '18',
 '18h',
 '19',
 '1er',
 '1jnat7v6lt',
 '1ère',
 '20',
 '200',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '21',
 '22',
 '23',
 '24',
 '25',
 '28',
 '29',
 '2emfoorogb',
 '2h',
 '2ème',
 '30',
 '31',
 '34',
 '3500',
 '40',
 '400',
 '42',
 '50',
 '500',
 '54',
 '60',
 '66',
 '70',
 '80',
 '90',
 '99',
 'abonnement',
 'abonnés',
 'abord',
 'abri',
 'absolument',
 'accord',
 'achat',
 'acheter',
 'acheté',
 'action',
 'activité',
 'actuellement',
 'adama',
 'adore',
 'affa',
 'affaire',
 'affaires',
 'afin',
 'afp',
 'afrique',
 'agit',
 'ah',
 'ai',
 'aide',
 'aider',
 'ailleurs',
 'aime',
 'aiment',
 'aimer',
 'aimerais',
 'aimes',
 'aimez',
 'aimé',
 'ainsi',
 'air',
 'ait',
 'al',
 'alarme',
 'album',
 'albums',
 'alcool',
 'alerte',
 'all',
 'allah',
 'aller',
 'allez',
 'alors',
 'ameeel',
 'ami',
 'amis',
 'amour',
 'amoureux',
 'amp',
 'an',
 'ancien',
 'and',
 'an

In [130]:
print(unicorpus[0,:])

  (0, 1351)	1
  (0, 837)	1
  (0, 172)	1
  (0, 1592)	1
  (0, 568)	1
  (0, 1583)	1
  (0, 170)	1
  (0, 344)	1


In [131]:
unicorpus.shape

(405160, 1717)

In [137]:
unigrams.index('rt')

1351

In [41]:
lda_model = lda.LDA(n_topics = 10)

In [42]:
lda_model.fit(unicorpus)

INFO:lda:n_documents: 405160
INFO:lda:vocab_size: 1717
INFO:lda:n_words: 5508855
INFO:lda:n_topics: 10
INFO:lda:n_iter: 2000
/Users/sumitsidana/anaconda3/lib/python3.6/site-packages/lda/utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):
INFO:lda:<0> log likelihood: -49198268
INFO:lda:<10> log likelihood: -37314194
INFO:lda:<20> log likelihood: -35262390
INFO:lda:<30> log likelihood: -34408764
INFO:lda:<40> log likelihood: -34008481
INFO:lda:<50> log likelihood: -33807183
INFO:lda:<60> log likelihood: -33716505
INFO:lda:<70> log likelihood: -33664773
INFO:lda:<80> log likelihood: -33629824
INFO:lda:<90> log likelihood: -33600894
INFO:lda:<100> log likelihood: -33578010
INFO:lda:<110> log likelihood: -33563485
INFO:lda:<120> log likelihood: -33545894
INFO:lda:<130> log likelihood: -33538811
INFO

INFO:lda:<1870> log likelihood: -33384936
INFO:lda:<1880> log likelihood: -33386229
INFO:lda:<1890> log likelihood: -33389440
INFO:lda:<1900> log likelihood: -33383866
INFO:lda:<1910> log likelihood: -33377075
INFO:lda:<1920> log likelihood: -33363686
INFO:lda:<1930> log likelihood: -33371542
INFO:lda:<1940> log likelihood: -33375270
INFO:lda:<1950> log likelihood: -33369721
INFO:lda:<1960> log likelihood: -33371110
INFO:lda:<1970> log likelihood: -33369066
INFO:lda:<1980> log likelihood: -33366423
INFO:lda:<1990> log likelihood: -33363974
INFO:lda:<1999> log likelihood: -33367657


In [ ]:
lda_model.

In [43]:
doc_topic = lda_model.doc_topic_

In [60]:
len(doc_topic)

405160

In [56]:
topic_word = lda_model.topic_word_  # model.components_ also works
n_top_words = 8
for i, topic_dist in enumerate(topic_word):
    topic_word_indexes = [np.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_words = np.array(unigrams)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))
    print(topic_word_indexes)

Topic 0: les rt tu que unk et on de
[array([   0, 1076, 1074, ..., 1572, 1351,  837])]
Topic 1: unk vous les rt de des pas qui
[array([   0,  953,  952, ...,  837, 1657, 1592])]
Topic 2: unk co de la le pour sur rt
[array([   0, 1181, 1180, ...,  430,  344, 1592])]
Topic 3: de la unk co le et les des
[array([   0,  499, 1442, ..., 1592,  820,  430])]
Topic 4: unk est il rt le on qu pas
[array([1716,  735,  736, ...,  745,  568, 1592])]
Topic 5: unk co est rt de le en hui
[array([   0, 1081, 1079, ...,  568,  344, 1592])]
Topic 6: unk de co le la pour en du
[array([ 858, 1017, 1015, ...,  344,  430, 1592])]
Topic 7: unk rt je quand tu que pas est
[array([   0, 1078, 1076, ...,  782, 1351, 1592])]
Topic 8: je unk rt de ai que la un
[array([   0,  949,  948, ..., 1351, 1592,  782])]
Topic 9: unk co rt est il de un la
[array([   0, 1080, 1079, ..., 1351,  344, 1592])]


In [58]:
print(topic_word.shape)

(10, 1717)


In [46]:
len(fr_val_input)

405160